In [3]:
#%% Import
from mednickdb_pysleep import sleep_features, pysleep_defaults, pysleep_utils
# Until pyparse has been merged into pysleep, you will need the latest version of pyparse:
# >>pip install git+https://bdyetton@bitbucket.org/mednicklab/mednickdb_pyparse.git
from mednickdb_pyparse import pyparse_utils, parse_scorefile  
import glob
import pandas as pd

In [4]:
#%% Find set of sleep files
drug = 'PBOPBO'
working_dir = 'C:/Users/bdyet/Desktop/Test_for_jing/'
edf_filepaths = glob.glob(working_dir + '*.edf')
channels_to_consider = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2', 'P3', 'P4', 'FP1', 
                        'FP2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'FT9', 'FT10', 'Fz', 'Cz', 
                        'CPz', 'PZ', 'POz', 'FPz']

drug_conts = {'pbo':((721, 3), (723, 4), (725, 2), (726, 2), (727, 3), (728, 4), (729, 1), (733, 1), (734, 1), (740, 2), (741, 4),
 (742, 2), (744, 2), (745, 2), (746, 4), (747, 1), (748, 1), (750, 2), (752, 1), (753, 4), (754, 1), (757, 4), (758, 2), (759, 4),
 (761, 2), (762, 4), (1731, 4), (1732, 1)), 
              'zol':((721, 1), (723, 2), (725, 4), (726, 1), (727, 1), (728, 1), (729, 3), (733, 3), (734, 4), (740, 3), (741, 2), (742, 1),
 (744, 4), (745, 3), (746, 1), (747, 2), (748, 2), (750, 1), (752, 4), (753, 3), (754, 4), (757, 3), (758, 3), (759, 2), (761, 3), (762, 1),
 (1731, 3), (1732, 3))}



In [15]:
#%% loop through edf's, extract spindles and then features of spindles (per stage)
spindle_features_cont = []
for edf_filepath in edf_filepaths:
    visitid = int(edf_filepath.split('_V')[-1].split('.')[0])
    subjectid = int(edf_filepath.split('_V')[-2].split('_')[-1])
    for drug, sub_cond in drug_conts.items():
        if (subjectid, visitid) in sub_cond:
            drugid = drug
            break
    else:
        print('Drug condition not found?!')
    
    #%%Get stage info
    scoring_filepath = edf_filepath.replace('.edf', '.mat')
    stagemap = pyparse_utils.get_stagemap_by_studyid(scoring_filepath, None)
    stagedata = parse_scorefile.parse_scorefile(scoring_filepath, stagemap)
    epochstage = stagedata['epochstage']   
    
    #%% Quartile calc
    try:
        start_sleep_epoch = len(epochstage) - epochstage[::-1].index('wbso')
    except ValueError:
        start_sleep_epoch = 0
    try:
        end_sleep_epoch = epochstage.index('wase')-1
    except ValueError:
        end_sleep_epoch = len(epochstage)
        
    total_sleep_epochs = end_sleep_epoch - start_sleep_epoch
    epochs_per_quartile = total_sleep_epochs/4
    quartiles = [(round(i*epochs_per_quartile), 
                  min(round(i*epochs_per_quartile+epochs_per_quartile), total_sleep_epochs))
                 for i in range(4)]
    
    print('subject slept', total_sleep_epochs*30/60/60, 'hrs')
    
    #%% actual spindle stuffs
    for q_idx, quartile in enumerate(quartiles):
        print('Detecting quartile', q_idx+1, 'spindles for', edf_filepath.split('\\')[-1])
        spindles = sleep_features.detect_spindles(edf_filepath, algo='Ferrarelli2007', 
                                                  chans_to_consider=channels_to_consider,
                                                  start_time=quartile[0]*pysleep_defaults.epoch_len, 
                                                  end_time=quartile[1]*pysleep_defaults.epoch_len)
        spindles['onset'] = spindles['onset'] - spindles['onset'][0] #reset because otherwise stages will all be off
        spindles = sleep_features.assign_stage_to_feature_events(spindles, epochstage[quartile[0]:quartile[1]])
        print('Detected', spindles.shape[0], 'spindles')
        spindle_features = sleep_features.sleep_feature_variables_per_stage(spindles, 
                                                                            epochstage[quartile[0]:quartile[1]], 
                                                                            stages_to_consider=['stage2', 'sws'], 
                                                                            av_across_channels=False)
        if spindle_features is None:
            continue
        spindle_features['subjectid'] = subjectid
        spindle_features['quartile'] = q_idx+1
        spindle_features['drug'] = drugid
        spindle_features_cont.append(spindle_features)
        print('Extracted features')

C:\Users\bdyet\GoogleDrive\mednickdb_pysleep\mednickdb_pysleep\sleep_dynamics.py:53: RuntimeWarning: invalid value encountered in true_divide
  second.astype(int)/np.expand_dims(np.sum(second, axis=2), 2)


subject slept 9.858333333333333 hrs
Detecting quartile 1 spindles for PSTIM_721_V3.edf


     freq_peak        onset  peak_time  peak_uV_in_spindle_band chan  \
0     9.968847     0.000000   1.406250                19.475875   O2   
1    11.505618     2.867188   0.230469                15.790374  FPz   
2    31.695238     6.937500   0.191406                22.793598  FP2   
3    33.259843    10.699219   0.207031                24.755338   O1   
4    47.169946    14.625000   1.472656                23.475782   F3   
5    11.064083    17.941406   0.464844                24.502246  POz   
6    10.847458    25.085938   0.804688                15.293027   T7   
7    11.725191    26.957031   0.687500                16.752331   P8   
8    11.347518    34.320312   0.746094                29.920062  POz   
9    11.264000    46.363281   0.578125                16.018627  FPz   
10   33.444181    56.621094   1.140625                31.977048  FPz   
11   10.824524    63.617188   0.398438                17.875662   O1   
12   11.252747    70.906250   0.824219                29.308151 

     freq_peak        onset  peak_time  peak_uV_in_spindle_band chan  \
0    11.130435     0.000000   0.292969                14.153231  FP1   
1    11.695431    19.417969   0.406250                17.232921  FT9   
2    13.544974    34.507812   0.511719                20.125572   PZ   
3    13.983193    52.890625   0.523438                21.752911   P3   
4    13.158879    56.242188   0.460938                21.250516  POz   
5    12.293785    61.742188   1.183594                15.566966  FP2   
6    13.913043    80.480469   0.503906                25.835854  CPz   
7    13.755224    92.789062   0.234375                21.228445   P3   
8    11.294118    96.355469   0.242188                17.759844   P7   
9    13.617021    99.011719   0.476562                35.032943   P3   
10   13.473684   115.722656   0.843750                28.501972  CPz   
11   11.349754   121.535156   0.199219                17.426576   F4   
12   12.137931   141.957031   0.753906                14.323812 

     freq_peak        onset  peak_time  peak_uV_in_spindle_band  chan  \
0    13.473684     0.000000   0.597656                18.317495    Cz   
1    12.296943     5.429688   0.527344                19.622110    F4   
2    13.342020    24.605469   0.636719                20.532306    PZ   
3    13.755224    37.515625   0.710938                19.009376   POz   
4    13.592920    41.386719   0.558594                23.947067    PZ   
5    13.913043    51.792969   0.359375                21.413580    P3   
6    11.695431    69.929688   0.250000                15.511948    Fz   
7    13.426573    86.937500   0.480469                22.889579    Cz   
8    11.946667    98.343750   0.300781                13.927345    F7   
9    12.118343   116.367188   0.476562                15.563358   FT9   
10   13.736585   120.445312   0.605469                20.401593    C3   
11   13.948718   135.796875   0.515625                23.568077    P3   
12   12.273973   140.218750   0.324219             

...

     freq_peak        onset  peak_time  peak_uV_in_spindle_band  chan  \
0    11.335793     0.000000   0.621094                30.958818    O2   
1     5.981308    54.960938   1.281250                22.904613    T7   
2    11.906977   122.992188   0.195312                25.803471    P7   
3    11.219124   129.863281   0.562500                37.845466    O2   
4    11.946667   142.875000   0.390625                33.092454    O2   
5    11.545098   251.125000   1.273438                46.893235    F4   
6    11.482850   261.203125   0.976562                35.303982   FP2   
7    14.000000   265.078125   0.917969                48.317551    PZ   
8    11.636364   279.878906   1.078125                24.960777    T7   
9    11.422572   289.300781   1.519531                35.327631   FP1   
10   11.600000   295.863281   1.656250                51.241481    F3   
11   13.661253   300.593750   0.988281                29.049921    O2   
12   11.752495   305.734375   1.359375             

     freq_peak        onset  peak_time  peak_uV_in_spindle_band chan  \
0    11.885714     0.000000   0.406250                31.608884   F4   
1    13.977011    15.902344   0.601562                27.174730  POz   
2    11.548872    25.945312   1.078125                23.865013  FPz   
3    13.698997    28.980469   0.777344                30.463256   PZ   
4    11.130435    35.906250   1.218750                33.916691   F4   
5    14.065934    42.480469   0.589844                22.663048   P8   
6    11.953307    46.132812   0.890625                21.694566   P8   
7    11.636364    50.339844   0.398438                15.965213  FT9   
8    14.054902    70.652344   0.820312                28.453563   P3   
9    13.685535    78.351562   0.773438                26.695193   O1   
10    7.165394    89.484375   1.109375                34.052964   F3   
11   13.381818    94.652344   0.554688                40.520400   O1   
12   14.048780   104.933594   0.191406                29.188453 

     freq_peak        onset  peak_time  peak_uV_in_spindle_band  chan  \
0    12.300300     0.000000   0.867188                35.396074    F4   
1     2.089796    14.378906   0.496094                79.391328    P3   
2    32.864865    15.593750   0.382812                16.009247  FT10   
3    11.243243    16.863281   0.351562                30.527575    Fz   
4    11.462687    18.910156   0.882812                30.181039    O1   
5    32.000000    21.109375   0.300781                24.381234    F8   
6    28.444444    21.574219   0.175781                29.514207    F8   
7     2.675958    24.707031   0.921875                59.015630    P3   
8    45.076412    25.832031   0.417969                16.805321  FT10   
9    23.741935    29.636719   0.589844               190.971976    P3   
10    4.357447    35.226562   1.375000              1382.473807   CPz   
11   37.063973    43.566406   0.738281                44.273700    P4   
12   12.487805    55.785156   0.199219             

     freq_peak        onset  peak_time  peak_uV_in_spindle_band chan  \
0     2.639175     0.000000   0.195312                36.440202  FT9   
1    11.475862    11.128906   1.468750               338.381929  FT9   
2     2.010471   339.765625   0.691406               286.796781   P3   
3     2.015748   341.343750   0.117188                39.653568   P3   
4     1.777778   342.234375   0.148438                49.764556   P3   
5     4.721311   612.042969   1.003906               298.547264   P3   
6     6.887892   614.023438   0.492188                58.699341  CPz   
7    38.524272   965.105469   0.460938                48.698994   T8   
8     6.522293   968.882812   0.410156                34.847092   O2   
9     6.481013   971.421875   0.191406                37.040584   PZ   
10    1.458689   972.183594   0.957031               110.485715   Cz   
11    8.311688   974.734375   0.832031                28.164199   C3   
12    2.485437   976.300781   0.105469                76.149266 

Extracted features


In [16]:
#Combine all spindles into a single dataframe and save to csv
all_spindles_info = pd.concat(spindle_features_cont, axis=0)
all_spindles_info.to_csv('PSTIM_'+drug+'_spindle_data.csv', index=False)

In [27]:
#%% Create datacube for matlab save:
chan_order = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2', 'P3', 'P4', 'Fp1', 'Fp2', 'F7',
              'F8', 'T7', 'T8', 'P7', 'P8', 'FT9', 'FT10', 'Fz', 'Cz', 'CPz', 'Pz', 'POz', 'Fpz']

subjects_order = [721, 723, 725, 726, 727, 728,  729, 733, 734, 740, 741, 742, 744,
                  745, 746, 747, 748, 750, 752, 753, 754, 757, 758, 759, 761, 762, 1731, 1732]

stage_order = ['stage2', 'sws']
drug_order = ['pbo']

dim_cols = ['drug', 'stage', 'quartile', 'chan', 'subjectid']
val_cols = set(all_spindles_info.columns) - set(dim_cols)
for val_col in val_cols:
    df = all_spindles_info
    df = df.loc[:, dim_cols+[val_col]].set_index(dim_cols)
    df = df.loc[:, val_col]
    dc = df.to_xarray()
    dc_ordered = dc.sel(drug=drug_order, 
                        stage=stage_order,
                        chan=chan_order,
                        subjectid=subjects_order
                        )
    pysleep_utils.data_to_matfile(dc_ordered, val_col+'_dc.mat')


In [23]:
df

drug  stage   quartile  chan  subjectid
pbo   stage2  1         C3    721          3
                        C4    721          3
                        CPz   721          3
                        Cz    721          3
                        F3    721          3
                        F4    721          3
                        F7    721          3
                        FP1   721          3
                        FP2   721          3
                        FPz   721          3
                        FT10  721          3
                        FT9   721          3
                        Fz    721          3
                        O1    721          3
                        P3    721          3
                        P4    721          3
                        P7    721          3
                        P8    721          3
                        POz   721          3
                        PZ    721          3
                        T7    721          3
      sws     1

In [21]:
df

drug  stage   quartile  chan  subjectid
pbo   stage2  1         C3    721          3
                        C4    721          3
                        CPz   721          3
                        Cz    721          3
                        F3    721          3
                        F4    721          3
                        F7    721          3
                        FP1   721          3
                        FP2   721          3
                        FPz   721          3
                        FT10  721          3
                        FT9   721          3
                        Fz    721          3
                        O1    721          3
                        P3    721          3
                        P4    721          3
                        P7    721          3
                        P8    721          3
                        POz   721          3
                        PZ    721          3
                        T7    721          3
      sws     1